In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)

In [ ]:
IMAGE_SIZE = 256
CHANNELS = 3
EPOCHS = 30

Data generators for tomato leaves


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    'dataset/tomato_train',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode="binary"  # Binary classification: tomato leaf or not
)
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    horizontal_flip=True
)
validation_generator = validation_datagen.flow_from_directory(
    'dataset/tomato_val',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode="binary"
)
test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    horizontal_flip=True
)
test_generator = test_datagen.flow_from_directory(
    'dataset/tomato_test',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode="binary"
)

Visualizing some images from the training set


In [ ]:
plt.figure(figsize=(15, 15))
images, labels = next(train_generator)
for i in range(12):
    ax = plt.subplot(3, 4, i + 1)
    plt.imshow(images[i])
    actual_class = "Tomato" if labels[i] == 1 else "Not Tomato"
    plt.title(f"Actual: {actual_class}", fontsize=12)
    plt.axis("off")
plt.tight_layout(pad=2.0)
plt.show()

 Model architecture

In [ ]:
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid'),  # Sigmoid for binary classification
])
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Binary crossentropy for binary classification
    metrics=['accuracy']
)
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=EPOCHS,
)


 Plotting training history

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Testing the model

In [ ]:
for image_batch, label_batch in test_generator:
    first_image = image_batch[0]
    first_label = label_batch[0]
    
    print("First image to predict")
    plt.imshow(first_image)
    actual_class = "Tomato" if first_label == 1 else "Not Tomato"
    print("Actual label:", actual_class)
    
    batch_prediction = model.predict(image_batch)
    predicted_class = "Tomato" if batch_prediction[0] > 0.5 else "Not Tomato"
    print("Predicted label:", predicted_class)
    
    break

 Save the model


In [ ]:
model.save("../saved_models/tomato_leaf_detector.keras")